In [1]:
import requests
import sys
import pandas as pd
import numpy as np
import time
import re
# time.sleep(5)
# page = requests.get('https://www.boxofficemojo.com/release/rl1182631425/?ref_=bo_yld_table_1')
# page_list = requests.get('https://www.boxofficemojo.com/year/2020/?grossesOption=calendarGrosses')
from bs4 import BeautifulSoup
# soup = BeautifulSoup(page.content, 'html.parser')
# print(soup.prettify())
page = requests.get('https://www.boxofficemojo.com/release/rl2960360961/?ref_=bo_yld_table_16')
# page = requests.get('https://www.boxofficemojo.com/release/rl101418497/?ref_=bo_yld_table_235')
time.sleep(1)
# soup = BeautifulSoup(page.content, 'html.parser')    

In [2]:
# returns an array of URLs of web page to scrape from 
def create_five_years_urls():
    url_list = []
    years = list(range(2016, 2021))
    for year in years:
        url_list.append(f'https://www.boxofficemojo.com/year/{year}/?grossesOption=calendarGrosses')
    return url_list

In [3]:
created_list_movies_urls = create_five_years_urls()

In [4]:
# returns the title of the single movie from single movie url in string form
def find_movie_title(page):
    # page = requests.get('https://www.boxofficemojo.com/release/rl2960360961/?ref_=bo_yld_table_16')
    # time.sleep(1)
    soup = BeautifulSoup(page.content, 'html.parser')    
    # print(soup.prettify())
    title = soup.find('h1', class_="a-size-extra-large")
    return title.string

In [5]:
find_movie_title(page)

'Star Trek Beyond'

In [6]:
# returns release date of the single movie in string
def find_release_date(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    release_date = soup.find_all('a', class_='a-link-normal')
    dates = []
    # print(release_date)
    for item in release_date:
        # print(item)
        if item['href'][-8:] == 'bo_rl_rl':
            # print('here')
            date = item.text
            # print(date)
            dates.append(item.text)
    if len(dates) >= 1:
        return dates[0]

In [7]:
find_release_date(page)

'Jul 22, 2016'

In [8]:
# returns budget cost or NaN of a movie if no budget cost is listed
def find_budget(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    budget = str(soup.find_all('div', class_='a-section a-spacing-none'))
    # print(budget)
    re_budget = re.findall(r"(?<=Budget</span><span><span class=\"money\">).*?(?=</span>)", budget)
    if re_budget:
        return re_budget[0]
    # print(budget)
    return float('NaN')


In [9]:
find_budget(page)

'$185,000,000'

In [10]:
# returns an array of genres of single movie
def find_genre(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    span_genre = str(soup.find_all('span')).replace('\n', '')
    span_genre = re.findall(r"(?<=<span>Genres<\/span>, <span>).*?(?=</span>)", span_genre)
    if span_genre:
        return span_genre[0].split()
    return []

In [11]:
find_genre(page)

['Action', 'Adventure', 'Sci-Fi', 'Thriller']

In [12]:
# returns gross value (domestic and world wide) in dict with two key-values
def find_all_gross(page):
    output = []
    soup = BeautifulSoup(page.content, 'html.parser') 
    all_money = soup.select('.a-link-normal > .money')
    if (len(all_money) == 2):
        international = int(all_money[0].text.replace('$', '').replace(',', ''))
        worldwide = int(all_money[1].text.replace('$', '').replace(',', ''))
        domestic = worldwide - international
        return {'domestic_gross': domestic, 'worldwide_gross': worldwide}
    if (len(all_money) < 2):
        worldwide = int(all_money[0].text.replace('$', '').replace(',', ''))
        return {'domestic_gross': worldwide, 'worldwide_gross': worldwide}
    else:
        return {'domestic_gross': float('NaN'), 'world_gross': float('NaN')}

In [13]:
find_all_gross(page)

{'domestic_gross': 158848340, 'worldwide_gross': 343471816}

In [14]:
# returns MPAA rating in string if not found, return nan
def find_mpaa(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    # print(soup)
    mpaa_rating = str(soup.find_all('div', class_='a-section a-spacing-none'))
    mpaa_rating = re.findall(r"(?<=<span>MPAA</span><span>).*?(?=</span>)", mpaa_rating)
    # print(mpaa_rating)
    if mpaa_rating:
        return mpaa_rating[0]
    return float('nan')

In [15]:
find_mpaa(page)

'PG-13'

In [16]:
def find_runtime(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    # print(soup.prettify())
    runtime = str(soup.select('.a-section .a-spacing-none'))
    runtime = re.findall(r"(?<=<div class=\"a-section a-spacing-none\"><span>Running Time</span><span>).*?(?=</span>)", runtime)
    if runtime:
        return runtime[0]
    return float('nan')
    # print(runtime)
    

In [17]:
find_runtime(page)

'2 hr 2 min'

In [18]:
# create a function that will return a dictionary of a single movie with keys as 'title', 'release_date', 
# 'genres', 'mpaa', 'budget_cost', 'total_gross'
def create_single_movie_dict(single_movie_url):
    new_movie = {}
    # print(single_movie_url)
    time.sleep(.5)

    page = requests.get(single_movie_url)
    # print('here', page.content)
    soup = BeautifulSoup(page.content, 'html.parser')    
    # print(soup.prettify())
    title = find_movie_title(page)
    # print(title)
    release_date = find_release_date(page)
    # print(release_date)
    genres = find_genre(page)
    # print(genres)
    mpaa = find_mpaa(page)
    # print(mpaa)
    budget_cost = find_budget(page)
    # print(budget_cost)
    all_gross = find_all_gross(page)
    # print(all_gross)
    runtime = find_runtime(page)
    # print(runtime)
    # assign into dict
    if title not in  new_movie:
        new_movie['title'] = title
        # print(title)
        new_movie['release_date'] = release_date
        new_movie['genres'] = genres
        new_movie['mpaa'] = mpaa
        new_movie['budget_cost'] = budget_cost
        new_movie['total_gross'] = all_gross
        new_movie['runtime'] = runtime
        
    return new_movie

In [19]:
create_single_movie_dict('https://www.boxofficemojo.com/release/rl2747040257/weekend/')

{'title': 'Moondance',
 'release_date': 'Mar 13, 2020',
 'genres': ['Musical'],
 'mpaa': nan,
 'budget_cost': nan,
 'total_gross': {'domestic_gross': 2371, 'worldwide_gross': 2371},
 'runtime': '1 hr 37 min'}

In [20]:
# takes in list of yearly movie urls and returns all single movie hrefs in a new list
def get_href_movie(urls):
    title_movies = []
    for link in urls:
        
        time.sleep(.5)
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        # print(soup.prettify())
        tb = soup.find_all('td', class_='a-text-left mojo-field-type-release mojo-cell-wide')
        # print(tb)
        for movie in tb:
            # print(movie.find_all('a')[0]['href'])
            end_url = movie.find_all('a')[0]['href']
            single_movie_url = f"https://www.boxofficemojo.com{end_url}"
        
            title_movies.append(single_movie_url)
    return title_movies

In [21]:
five_year_movies_hrefs = get_href_movie(created_list_movies_urls)
five_year_movies_hrefs

['https://www.boxofficemojo.com/release/rl3764946433/?ref_=bo_yld_table_1',
 'https://www.boxofficemojo.com/release/rl2557707777/?ref_=bo_yld_table_2',
 'https://www.boxofficemojo.com/release/rl3210970625/?ref_=bo_yld_table_3',
 'https://www.boxofficemojo.com/release/rl2069595649/?ref_=bo_yld_table_4',
 'https://www.boxofficemojo.com/release/rl2455602689/?ref_=bo_yld_table_5',
 'https://www.boxofficemojo.com/release/rl2588706305/?ref_=bo_yld_table_6',
 'https://www.boxofficemojo.com/release/rl4031743489/?ref_=bo_yld_table_7',
 'https://www.boxofficemojo.com/release/rl2238875137/?ref_=bo_yld_table_8',
 'https://www.boxofficemojo.com/release/rl1145865729/?ref_=bo_yld_table_9',
 'https://www.boxofficemojo.com/release/rl2691925505/?ref_=bo_yld_table_10',
 'https://www.boxofficemojo.com/release/rl3076752897/?ref_=bo_yld_table_11',
 'https://www.boxofficemojo.com/release/rl3696264705/?ref_=bo_yld_table_12',
 'https://www.boxofficemojo.com/release/rl4249847297/?ref_=bo_yld_table_13',
 'https:

In [22]:
# create function that will iterate over all hrefs
# and that will return a single movie dictionary with title, release date, genre, MPAA rating, budget cost, gross
# input: list of single movie url
# output: dict of single movie

def create_all_movies_list(single_movie_urls):
    movies = []
    for single_movie in single_movie_urls:
        time.sleep(.5)
        
        movies.append(create_single_movie_dict(single_movie))
    return movies

In [23]:
movies_data_list = create_all_movies_list(five_year_movies_hrefs)

In [24]:
movie_df = pd.DataFrame(movies_data_list)
movie_df.to_csv('movie_data.csv')

In [25]:
movie_df

,title,release_date,genres,mpaa,budget_cost,total_gross,runtime
0,Finding Dory,"Jun 17, 2016","[Adventure, Animation, Comedy, Family]",PG,NaN,"{'domestic_gross': 486295561, 'worldwide_gross...",1 hr 37 min
1,Rogue One: A Star Wars Story,"Dec 16, 2016","[Action, Adventure, Sci-Fi]",PG-13,"$200,000,000","{'domestic_gross': 532177324, 'worldwide_gross...",2 hr 13 min
2,Captain America: Civil War,"May 6, 2016","[Action, Adventure, Sci-Fi]",PG-13,"$250,000,000","{'domestic_gross': 408084349, 'worldwide_gross...",2 hr 27 min
3,The Secret Life of Pets,"Jul 8, 2016","[Adventure, Animation, Comedy, Family]",PG,"$75,000,000","{'domestic_gross': 368384330, 'worldwide_gross...",1 hr 27 min
4,The Jungle Book,"Apr 15, 2016","[Adventure, Drama, Family, Fantasy]",PG,"$175,000,000","{'domestic_gross': 364001123, 'worldwide_gross...",1 hr 46 min
...,...,...,...,...,...,...,...
3872,I Do Not Care If We Go Down in History as Barb...,"Jul 19, 2019","[Comedy, Drama]",NaN,NaN,"{'domestic_gross': 11307, 'worldwide_gross': 3...",2 hr 20 min
3873,The Load,"Aug 30, 2019",[Drama],NaN,NaN,"{'domestic_gross': 28410, 'worldwide_gross': 2...",1 hr 38 min
3874,Asako I & II,"May 17, 2019","[Drama, Romance]",NaN,NaN,"{'domestic_gross': 25559, 'worldwide_gross': 6...",1 hr 59 min
3875,Chained for Life,"Sep 13, 2019",[Drama],NaN,NaN,"{'domestic_gross': 17431, 'worldwide_gross': 1...",1 hr 31 min
